## - - 이전 코드 시작 - -

In [2]:
!pip install ratsnlp

     |████████████████████████████████| 42 kB 1.0 MB/s 
     |████████████████████████████████| 57 kB 6.3 MB/s 
     |████████████████████████████████| 806 kB 55.1 MB/s 
     |████████████████████████████████| 2.8 MB 60.1 MB/s 
     |████████████████████████████████| 829 kB 49.3 MB/s 
     |████████████████████████████████| 136 kB 71.5 MB/s 
     |████████████████████████████████| 636 kB 55.5 MB/s 
     |████████████████████████████████| 398 kB 56.3 MB/s 
     |████████████████████████████████| 3.3 MB 48.9 MB/s 
     |████████████████████████████████| 895 kB 46.6 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 1.1 MB 47.8 MB/s 
     |████████████████████████████████| 96 kB 6.2 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 271 kB 78.0 MB/s 
     |████████████████████████████████| 144 kB 78.4 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sh

In [3]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [4]:
import torch
from ratsnlp.nlpbook.qa import QATrainArguments
args = QATrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="korquad-v1",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-qa",
    max_seq_length=128,
    max_query_length=32,
    doc_stride=64,
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

In [5]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [6]:
nlpbook.set_logger(args)

In [7]:
nlpbook.download_downstream_dataset(args)

Downloading: 38.5MB [00:00, 79.1MB/s]
Downloading: 3.88MB [00:00, 77.8MB/s]                  


In [8]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

## - - 이전 코드 끝 - -

<**4단계**> **토크나이저 준비하기**    

다음 코드를 실행해 `kcbert-base` 모델이 사용하는 토크나이저를 선언합니다.

In [9]:
#토크나이저 준비
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

<**5단계**> **데이터 전처리하기**   

딥러닝 모델을 학습하려면 데이터 로더를 이용해 데이터를 배치 단위로 모델에 공급해 주어야 합니다.  
다음 코드를 실행하면 학습용 데이터 로더의 구성 요소인 데이터셋(`train_dataset`)이 만들어집니다.

In [10]:
#학습 데이터셋 구축
from ratsnlp.nlpbook.qa import KorQuADV1Corpus, QADataset
corpus = KorQuADV1Corpus()
train_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

convert squad examples to features: 100%|██████████| 57688/57688 [07:34<00:00, 127.00it/s]


`KorQuADV1Corpus` 클래스는 JSON 포맷의 KorQuAD 1.0 데이터를 `QAExample`로 읽어옵니다.  
`KorQuADV1Corpus`는 `QADataset`이 요구하면 `QAExample`을 `QADataset`에 제공합니다. 0번 `QAExample`은 다음과 같습니다.

- **Question_text(질문)**: 바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?
- **Context_text(지문)**: 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.(…생략…)  
- **answer_text(정답)**: 교향곡
- **start_position_character(정답의 시작 인덱스)**: 54

참고로 파이썬에서는 어떤 요소의 순서를 부여할 때 0부터 시작합니다. 만일 `start_position_character`가 0이라면 `context_text`의 첫 번째 글자가 정답의 시작이라는 뜻입니다. 마찬가지로 `start_position_character`가 54라면 정답 시작 글자는 쉰다섯 번째가 됩니다. 이처럼 순서를 셀 때는 공백, 줄바꿈 등도 글자 하나로 취급합니다.

`QADataset` 클래스는 `tokenizer`를 활용해 `KorQuADV1Corpus`가 넘겨준 `QAExample`을 모델이 학습할 수 있는 형태(`QAFeatures`)로 가공합니다.   
`QAFeatures` 각 구성 요소의 자료형은 다음과 같습니다.

- **input_ids**: List[int]
- **attention_mask**: List[int]
- **start_positions**: int
- **end_positions**: int

`KorQuADV1Corpus`가 넘겨준 0번 `QAExample`에 대한 `QADataset` 클래스의 중간 산출물은 다음과 같습니다.  
그 처리 과정은 이렇습니다. `QADataset` 클래스는 `QAExample`의 `question_text`를 질문, `context_text`을 지문으로 취급하고 '**[CLS] 질문 [SEP] 지문 [SEP]**'으로 이어붙인 뒤 토큰화를 수행합니다. 이때 이전 코드에서 입력 문장(질문+지문) 최대 길이인 `max_seq_length`와 질문 최대 길이인 `max_query_length`를 각각 128, 32로 설정해 두었으므로 전체 토큰 개수와 질문 토큰 개수가 이보다 많아지지 않도록 토큰화합니다.

그리고 `QAExample`의 `answer_text`와 `start_position_character`를 활용해 정답 시작과 끝 토큰을 찾습니다. 지금 예시에서 이에 해당하는 토큰은 각각 `교`, `##곡`입니다.

In [11]:
train_dataset[0]

QAFeatures(input_ids=[2, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 19143, 13985, 12449, 9194, 4105, 3385, 9411, 32, 3, 8601, 4633, 29697, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 4104, 4027, 8793, 13985, 391, 9132, 4113, 10966, 11728, 12023, 14657, 4091, 8598, 16639, 341, 4573, 4771, 4027, 2139, 8478, 14416, 214, 8202, 17, 2451, 13007, 1480, 4313, 4538, 4008, 8601, 4633, 22903, 4113, 1676, 868, 4913, 7965, 1789, 4203, 4110, 15031, 786, 250, 4057, 10878, 4007, 2593, 4094, 4128, 10289, 4113, 10958, 4062, 9511, 1355, 4600, 4103, 4775, 5602, 10770, 4180, 26732, 3231, 23243, 4104, 4042, 2015, 4012, 4113, 9198, 8763, 8129, 17, 10384, 23008, 7971, 2170, 4408, 4011, 4147, 4042, 17015, 4091, 23008, 21056, 4165, 323, 4175, 4158, 11413, 2273, 4043, 7966, 1543, 4775, 3], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

위 예시에서는 패딩 토큰이 전혀 없으므로 `attention_mask`가 모두 1인 걸 알 수 있습니다.   
`train_dataset[0].input_ids`는 원본 문장을 토큰화하나 뒤 인덱싱을 수행한 결과이며 `train_dataset[0].attention_mask`는 해당 위치 토큰이 패딩인지(`0`) 아닌지(`1`)를 나타냅니다.

`train_dataset[0].token_type_ids`는 세그먼트 정보를 나타냅니다. **[CLS] 질문 [SEP]**에 해당하는 첫 번째 세그먼트는 `0`, **지문 [SEP]**에 해당하는 두 번째 세그먼트는 `1`, 나머지 패딩에 속하는 세 번째 세그먼트는 `0`을 줍니다. 질문과 지문의 토큰 수는 각각 17개와 108개이므로 `0`으로 채우는 첫 번째 세그먼트의 길이는 **[CLS]와 [SEP]**를 합쳐 19개, `1`로 채우는 두 번째 세그먼트는 **[SEP]**를 포함해 109개가 됩니다. 마지막 세그먼트(`0`으로 채움)의 길이는 128(`max_seq_length`) - 19(첫 번째 세그먼트 길이) - 109(두 번째 세그먼트 길이), 즉 0이 됩니다.

한편 `train_dataset[0].start_positions`와 `train_dataset[0].end_positions`는 각각 `train_dataset[0].input_ids`에서 정답 토큰의 시작과 끝 인덱스를 가리킵니다. 다시 말해 정답의 시작 토큰인 `교`는 `train_dataset[0].input_ids`에서의 인덱스가 45, 끝 토큰인 `##곡`은 47이라는 뜻입니다.  

cf) start_positions가 0이라면 input_ids의 첫 번째가 정답의 시작이라는 뜻입니다.  
마찬가지로 start_positions가 45라면 정답 시작은 input_ids의 마흔여섯 번째가 됩니다.